In [ ]:
import os
from PIL import Image, ExifTags

def correct_image_orientation(img):
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = img._getexif()
        if exif is not None:
            orientation = exif.get(orientation, None)
            if orientation == 3:
                img = img.rotate(180, expand=True)
            elif orientation == 6:
                img = img.rotate(270, expand=True)
            elif orientation == 8:
                img = img.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # Cases where there is no EXIF data or orientation information
        pass
    return img

def create_low_res_image(input_path, output_path, scale_factor):
    with Image.open(input_path) as img:
        # Correct orientation based on EXIF data
        img = correct_image_orientation(img)
        
        # Calculate new dimensions
        new_width = int(img.width * scale_factor)
        new_height = int(img.height * scale_factor)
        
        # Resize the image
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        
        # Save the resized image
        img.save(output_path)

def process_directory(input_dir, output_dir, scale_factor):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif')):
                input_path = os.path.join(root, file)
                print(f"Attempting {input_path}")
                relative_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, relative_path)

                output_dir_for_file = os.path.dirname(output_path)
                if not os.path.exists(output_dir_for_file):
                    os.makedirs(output_dir_for_file)

                create_low_res_image(input_path, output_path, scale_factor)
                print(f'Converted {input_path} to {output_path}')

input_directory = "./"
output_directory = "./lowres/"
scale_factor = 0.5

process_directory(input_directory, output_directory, scale_factor)
